In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%fs ls

In [0]:
%fs ls dbfs:/databricks-datasets/asa/

1. Creating a dataframe using airlines csv datasets.

In [0]:
df=spark.read.option("header",True).csv("dbfs:/databricks-datasets/asa/airlines/*")

Displaying the newly create Dataframe

In [0]:
df.display()

2. Showing only 3 columns from the dataframe

In [0]:
df.select("FlightNum","ActualElapsedTime","CRSElapsedTime").display()

In [0]:
df.select("Origin","Dest","Distance").display()

3. Showing all the columns and their respective data types from Dataframe.

In [0]:
df.printSchema()

4.Creating a new dataframe with proper datatypes for different columns

In [0]:
n_df = spark.read.format("csv").option("inferSchema", True).option("header", True).load("dbfs:/databricks-datasets/asa/airlines/*")

 Showing all the columns and their respective data types from  the new Dataframe.

In [0]:
n_df.printSchema()

5. Rename few columns and then display

In [0]:
nw_df=n_df.withColumnRenamed("DepTime", "departure_time").withColumnRenamed("DayOfWeek", "weekday")

In [0]:
nw_df.display()

6.  Create a new dataframe excluding dropped column DayofMonth

In [0]:
new_df = nw_df.drop("DayofMonth")

In [0]:
new_df.display()

7. Adding a new column named "weekand" and adding value to it.If it is saturday or sunday then 1 otherwise 0

In [0]:
new_df = new_df.withColumn("Weekend", when((col("weekday") == 6) |(col("weekday") == 7), 1).otherwise(0))
# n_df = n_df.withColumn("Weekend", when(col("weekday").isin(6, 7), 1).otherwise(0))-----Another method of doing it.

In [0]:
new_df.display()

8. Cast "ActualElapsedTime" column to integer and use printSchema to verify

In [0]:
new_df.withColumn("ActualElapsedTime",new_df.ActualElapsedTime.cast(IntegerType())).printSchema()

9. Drop duplicate rows based on Year and Month and create new df.

In [0]:
new_df=new_df.dropDuplicates(["Year","Month"])

In [0]:
new_df.display()

10. Filtering Data with 'where' method

In [0]:
new_df.select('*').where(col("year")==2001).display()

In [0]:
new_df.where((col("DepTime") > 2000) & (col("ArrTime") >2000)).display()

11.  Group Data according to 'Origin' and count

In [0]:
new_df.groupBy("Origin").count().display()

12.Sort the data by 'Year' in descending order

In [0]:
new_df.sort(desc("Year")).display()

13. Write data in delta format

In [0]:
new_df.write.format("delta").save("/mnt//FileStore/tables/")